# Visualizing Var Rename Activations

In [ ]:
import torch
import numpy as np

from iluvattnshun.utils import load_checkpoint, load_config_from_yaml
from iluvattnshun.nn import MultilayerTransformer

from var_rename import VariableRenamingConfig, VariableRenamingPrompter
from iluvattnshun.viz import get_fig

In [2]:
config_path = "your config path..."
ckpt_path = "your checkpoint path..."

In [ ]:
config = load_config_from_yaml(config_path, VariableRenamingConfig)

# loading the model
max_seq_len = config.num_renames * 4
model = MultilayerTransformer(
    vocab_size=39,
    d_model=config.dim_model,
    n_heads=config.num_heads,
    n_layers=config.num_layers,
)
load_checkpoint(ckpt_path, model, None, "cuda")

# loading the prompter
prompter = VariableRenamingPrompter(config)

In [ ]:
# printing out the prompt, pred (per token), and true answer
rng = np.random.default_rng(42)
prompt, answer, metadata = prompter.get_prompt(rng)
x = torch.tensor(prompter.tokenize(prompt)).unsqueeze(0)
logits, attn_weights, _= model.forward(x, return_attn_weights=True)
pred = prompter.detokenize(logits[0].argmax(dim=-1).tolist())

print("prompt: ", prompt)
print("pred:   ", pred)
print("answer: ", answer)


In [7]:
# getting batchless np attn weights and plotting
weights = [attn_weight.detach().cpu().numpy()[0] for attn_weight in attn_weights]
plotly_fig = get_fig(weights, list(prompt))

In [ ]:
plotly_fig.show()

In [ ]:
# With matplotlib, plot attention weights over layers and heads

# token_labels = list(prompt)
# num_layers = len(attn_weights)
# num_heads = config.num_heads
# total_heads = num_layers * num_heads

# rows = num_layers
# cols = num_heads
# fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5), squeeze=False)

# head_idx = 0
# for layer_idx, attn_weight_heads in enumerate(attn_weights):
#     for head_number in range(num_heads):
#         row = head_idx // cols
#         col = head_idx % cols
#         ax = axes[row][col]

#         attn_weight = attn_weight_heads[0, head_number].detach().cpu().numpy()
#         im = ax.imshow(attn_weight, cmap="viridis")

#         ax.set_title(f"L{layer_idx}H{head_number}", fontsize=20)

#         ax.set_xticks(range(len(token_labels)))
#         ax.set_xticklabels(token_labels, fontsize=7)
#         ax.set_yticks(range(len(token_labels)))
#         ax.set_yticklabels(token_labels, fontsize=7)

#         head_idx += 1

# # Hide any unused subplots
# for i in range(head_idx, rows * cols):
#     fig.delaxes(axes[i // cols][i % cols])

# fig.suptitle("Attention Weights by Layer and Head", fontsize=25)
# plt.tight_layout(rect=[0, 0, 1, 0.95])
# plt.show()
